In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [13]:
df=pd.read_csv('/content/PoetryFoundationData.csv')
df.head()

,Unnamed: 0,Title,Poem,Poet,Tags
0,0,\r\r\n Objects Used to Prop...,"\r\r\nDog bone, stapler,\r\r\ncribbage board, ...",Michelle Menting,NaN
1,1,\r\r\n The New Church\r\r\n...,"\r\r\nThe old cupola glinted above the clouds,...",Lucia Cherciu,NaN
2,2,\r\r\n Look for Me\r\r\n ...,\r\r\nLook for me under the hood\r\r\nof that ...,Ted Kooser,NaN
3,3,\r\r\n Wild Life\r\r\n ...,"\r\r\nBehind the silo, the Mother Rabbit\r\r\n...",Grace Cavalieri,NaN
4,4,\r\r\n Umbrella\r\r\n ...,\r\r\nWhen I push your button\r\r\nyou fly off...,Connie Wanek,NaN


In [14]:
# find the null values
df.isnull().sum()

,0
Unnamed: 0,0
Title,0
Poem,0
Poet,0
Tags,955


In [15]:
print(df.describe())

         Unnamed: 0
count  13854.000000
mean      93.204417
std       57.493544
min        0.000000
25%       42.000000
50%       92.000000
75%      142.000000
max      199.000000


In [16]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13854 entries, 0 to 13853
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  13854 non-null  int64 
 1   Title       13854 non-null  object
 2   Poem        13854 non-null  object
 3   Poet        13854 non-null  object
 4   Tags        12899 non-null  object
dtypes: int64(1), object(4)
memory usage: 541.3+ KB
None


In [19]:
import re
corpus = "\n".join(df['Poet'].values)
corpus = corpus.lower()
corpus = re.sub(r'[^\w\s]', '', corpus)

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
total_words = len(tokenizer.word_index) + 1

# Convert text into sequences of integers
input_sequences = []
corpus_words = corpus.split()
for i in range(5, len(corpus_words)):
    sequence = corpus_words[i-5:i+1]
    tokenized_seq = tokenizer.texts_to_sequences([" ".join(sequence)])[0]
    input_sequences.append(tokenized_seq)

# Pad sequences
max_sequence_len = 5  # length of each sequence
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len + 1)

In [21]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]
X, y = X[:10000], y[:10000]
y = np.array(y)

In [22]:
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [25]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y, epochs=50, batch_size =128, verbose=1)

Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 10s 72ms/step - accuracy: 0.0555 - loss: 5.7795
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - accuracy: 0.0773 - loss: 5.5623
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 72ms/step - accuracy: 0.0904 - loss: 5.3475
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.1139 - loss: 5.1557
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.1393 - loss: 4.9488
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 77ms/step - accuracy: 0.1618 - loss: 4.7388
Epoch 7/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 0.1801 - loss: 4.5893
Epoch 8/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step - accuracy: 0.2097 - loss: 4.3840
Epoch 9/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.2232 - loss: 4.2679
Epoch 10/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - accuracy: 0.2302 - loss: 4.1495
Epoch 11/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.2637 - loss: 3.9456
Epoch 12/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - accuracy

In [29]:
def generate_poetry(seed_text, next_words=5000):
    generated_words = set()
    poem = seed_text

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([poem])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)

        next_word = tokenizer.index_word.get(predicted[0], None)
        if next_word is None or next_word in generated_words:
            continue

        generated_words.add(next_word)
        poem += " " + next_word

    return poem

print(generate_poetry("The Morning Sun Shine", next_words=500))


The Morning Sun Shine talamantez brolaski


In [31]:
# Generate multiple lines of poetry using different starting phrases
seed_texts = ["the moonlight whispers", "in the quiet of night", "stars shine brightly", "a gentle breeze flows", "echoes in silence"]

for seed in seed_texts:
    print(f"Seed: {seed}")
    print(generate_poetry(seed, next_words=50))  # Removed words_per_line
    print("\n" + "="*50 + "\n")

Seed: the moonlight whispers
the moonlight whispers xie


Seed: in the quiet of night
in the quiet of night hennessy stanley caconrad jim gibbons kathryn nuernberger warren


Seed: stars shine brightly
stars shine brightly wieners xie


Seed: a gentle breeze flows
a gentle breeze flows daniel johnson michael palmer


Seed: echoes in silence
echoes in silence wieners xie


